计算`Fama-French`三因子
+ 加无风险利率 -> 五因子
+ 使用run_pipeline生成因子数据

In [1]:
%load_ext zipline

In [2]:
%%zipline --start 2017-12-1 --end 2018-1-8 --capital-base 100000
from zipline.api import symbol, sid, get_datetime

import pandas as pd
import numpy as np
from zipline.api import attach_pipeline, pipeline_output
from zipline.pipeline import Pipeline
from zipline.pipeline import CustomFactor
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline import Fundamentals
#from zipline.pipeline.builtin import IsStock

# time frame on which we want to compute Fama-French
normal_days = 31
# approximate the number of trading days in that period
# this is the number of trading days we'll look back on,
# on every trading day.
business_days = int(0.69 * normal_days)

class Returns(CustomFactor):
    """
    每个交易日每个股票窗口长度"business_days"期间收益率
    """
    window_length = business_days
    inputs = [USEquityPricing.close]
    def compute(self, today, assets, out, price):
        out[:] = (price[-1] - price[0]) / price[0] * 100

class MarketEquity(CustomFactor):
    """
    每个交易日每只股票所对应的总市值
    """
    window_length = business_days
    inputs = [USEquityPricing.tmv]
    def compute(self,today,assets,out,mcap):
        out[:] = mcap[0]

class BookEquity(CustomFactor):
    """
    每个交易日每只股票所对应的账面价值（所有者权益）
    """
    window_length = business_days
    inputs = [Fundamentals.balance_sheet.A107]
    def compute(self, today, assets, out, book):
        out[:] = book[0]
                                        
class CommonStock(CustomFactor):
    """
    this factor outputs 1.0 for all securities that are either common stock or SPY,
    and outputs 0.0 for all other securities. This is to filter out ETFs and other
    types of share that we do not wish to consider.
    """
    window_length = business_days
    inputs = [USEquityPricing.cmv]
    def compute(self, today, assets, out, share_class):
        out[:] = (((share_class[-1] > 0. ).astype(bool)) | (assets == 100001)).astype(float)                                     
        
def initialize(context):
    """
    use our factors to add our pipes and screens.
    """
    pipe = Pipeline()
    common_stock = CommonStock()
    # 过滤：或是普通股或是上证指数
    pipe.set_screen(common_stock.eq(1))
    mkt_cap = MarketEquity()
    pipe.add(mkt_cap,'market_cap')
    
    book_equity = BookEquity()
    # book equity over market equity
    be_me = book_equity / mkt_cap
    pipe.add(be_me,'be_me')

    returns = Returns()
    pipe.add(returns,'returns')

    attach_pipeline(pipe, 'ff_example')

    
def before_trading_start(context,data):
    """
    every trading day, we use our pipes to construct the Fama-French
    portfolios, and then calculate the Fama-French factors appropriately.
    """
    # 区别于股票代码，指数代码在原始代码前+100000
    spy = sid(100001) # 上证指数
    
    factors = pipeline_output('ff_example')
    
    # get the data we're going to use
    returns = factors['returns']
    mkt_cap = factors.sort_values(['market_cap'], ascending=True)
    be_me = factors.sort_values(['be_me'], ascending=True)
    
    # to compose the six portfolios, split our universe into portions
    half = int(len(mkt_cap) * 0.5)
    small_caps = mkt_cap[:half]
    big_caps = mkt_cap[half:]
    
    thirty = int(len(be_me) * 0.3)
    seventy = int(len(be_me) * 0.7)
    growth = be_me[:thirty]
    neutral = be_me[thirty:seventy]
    value = be_me[seventy:]
    
    # now use the portions to construct the portfolios.
    # note: these portfolios are just lists (indices) of equities
    small_value = small_caps.index.intersection(value.index)
    small_neutral = small_caps.index.intersection(neutral.index)
    small_growth = small_caps.index.intersection(growth.index)
    
    big_value = big_caps.index.intersection(value.index)
    big_neutral = big_caps.index.intersection(neutral.index)
    big_growth = big_caps.index.intersection(growth.index)
    
    # take the mean to get the portfolio return, assuming uniform
    # allocation to its constituent equities.
    sv = returns[small_value].mean()
    sn = returns[small_neutral].mean()
    sg = returns[small_growth].mean()
    
    bv = returns[big_value].mean()
    bn = returns[big_neutral].mean()
    bg = returns[big_growth].mean()
    
    # computing Rm-Rf (Market Returns - Risk-Free Returns).  we take the
    # rate of risk-free returns to be zero, so this is simply SPY's returns.
    # have to set an initial dummy value
    context.rm_rf = float('nan')
    if spy in returns.index:
        context.rm_rf = returns.loc[spy]
    
    # computing SMB
    context.smb = (sv + sn + sg) / 3 - (bv + bn + bg) / 3
    
    # computing HML
    context.hml = (sv + bv) / 2 - (sg + bg) / 2

def handle_data(context, data):
    # print the Fama-French factors for the period defined by business_days
    # ending on the previous trading day.
    print(context.rm_rf, context.smb, context.hml)

-1.9543293267 -2.3517857534930293 3.5102941140154993
-1.60531167254 -2.4863309181971647 3.5549059412509534
-2.3185349985 -3.350271637125747 5.22358032874629
-3.21947026782 -3.716230306809363 5.807928944695156
-3.55723679973 -3.919716035733665 4.879748736661866
-4.54344689325 -3.5524940249977313 4.668638911273932
-4.15655671251 -3.0067411419977734 4.793917895115838
-3.64402588871 -2.8395574274558273 5.023976817476613
-4.33686888185 -3.107601130647626 4.695885208428474
-2.92391765642 -3.568561746116748 3.4543579205099313
-3.14222254909 -3.087429096282497 2.880847670043525
-3.45176500566 -1.2272480731062556 1.9901322905025856
-3.66926286361 -0.7688642762504965 3.5930307037254092
-3.34141915387 -0.01447976720164501 3.200477124006657
-4.16442207235 -0.3653228217862572 2.8199142636381276
-1.54714405305 -1.2495240339457752 1.5063865993591894
-1.69230906847 -1.4253190569410172 1.123391770729158
-1.25722820432 -1.8560441915592423 -0.1453767647743529
-0.825909804158 -1.2291139522909291 -0.324670

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ending_cash,ending_exposure,ending_value,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2017-12-01 07:00:00+00:00,NaN,0.0,NaN,-0.001988,NaN,NaN,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,1,[],0.0
2017-12-04 07:00:00+00:00,0.0,0.0,0.0,0.003185,0.080486,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,2,[],0.0
2017-12-05 07:00:00+00:00,0.0,0.0,0.0,0.008505,0.066277,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,3,[],0.0
2017-12-06 07:00:00+00:00,0.0,0.0,0.0,0.002427,0.088732,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,4,[],0.0
2017-12-07 07:00:00+00:00,0.0,0.0,0.0,-0.008747,0.113494,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,5,[],0.0
2017-12-08 07:00:00+00:00,0.0,0.0,0.0,-0.000679,0.120000,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,6,[],0.0
2017-12-11 07:00:00+00:00,0.0,0.0,0.0,0.015826,0.148077,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,7,[],0.0
2017-12-12 07:00:00+00:00,0.0,0.0,0.0,0.002475,0.162141,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,8,[],0.0
2017-12-13 07:00:00+00:00,0.0,0.0,0.0,0.010982,0.157652,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,9,[],0.0
2017-12-14 07:00:00+00:00,0.0,0.0,0.0,0.005005,0.152931,0.0,0.0,100000.0,0.0,0.0,...,0,0,0,None,100000.0,0.0,0.0,10,[],0.0
